In [174]:
import pandas as pd
import numpy as np
import re
from function_file import clean_names


In [175]:
raw_candy_15 = pd.read_excel(r"data_in\boing-boing-candy-2015.xlsx")
raw_candy_16 = pd.read_excel(r"data_in\boing-boing-candy-2016.xlsx")
raw_candy_17 = pd.read_excel(r"data_in\boing-boing-candy-2017.xlsx")

C:\Users\K\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [176]:
# raw_candy_16.columns
# raw_candy_17.columns

In [177]:
candy_15 = raw_candy_15.drop(['Timestamp'],axis = 1).copy()

# Rename Age column for consistency
candy_15_col_name= {'Are you going actually going trick or treating yourself?':'go_out',
    'How old are you?':'age'}
candy_15.rename(columns = candy_15_col_name, inplace = True)

#This is the pivot long equalvalent
candy_15 = candy_15.melt(id_vars = ['age', 'go_out'])

# Create core Columns required for this analysis
candy_15["country"] = np.nan
candy_15["gender"] = ""
candy_15["year"] = "2015"




In [178]:
# Drop columns
candy_16 = raw_candy_16.drop(['Timestamp'],axis = 1).copy()

# Create Year Column
candy_16["year"] = "2016"

candy_16_col_name= {'Are you going actually going trick or treating yourself?':'go_out',
                    'How old are you?':'age',
                    'Which country do you live in?': 'country',
                    'Your gender:':'gender'}
candy_17.drop(candy_17.filter(regex=':').columns, axis=1, inplace=True)
candy_16.rename(columns = candy_16_col_name, inplace = True)

#This is the pivot long equalvalent
candy_16 = candy_16.melt(id_vars = ['year', 'age', 'country', 'gender','go_out'])


In [179]:
# Drop columns
candy_17 = raw_candy_17.drop(['Internal ID', 'Q1: GOING OUT?', 'Q5: STATE, PROVINCE, COUNTY, ETC', 'Click Coordinates (x, y)'],axis = 1).copy()

# Create Year Column
candy_17["year"] = "2017"

# Rename core columns (gender, age, country)
candy_17_col_name = {'Q2: GENDER': "gender",
                     'Q3: AGE':"age",
                     'Q4: COUNTRY':"country",
                     'Q1: GOING OUT?':'go_out'}
candy_17.rename(columns=candy_17_col_name, inplace = True)



#Drop column with ':' (none candy columns)
candy_17.drop(candy_17.filter(regex=':').columns, axis=1, inplace=True)

#Remove "Q1 | " pattern for all column names
candy_17.rename(columns=lambda x: re.sub('([Qq].*? \\| )','',x), inplace=True)

candy_17 = candy_17.melt(id_vars = ['year', 'age', 'country', 'gender'])

In [180]:
# Bind dataframes for 2015,2016,2017
df = pd.concat([candy_15, candy_16, candy_17])

In [181]:
clean_country = df.copy()

clean_country["country"] = clean_country["country"].str.upper()

#Remove brackets and whats inside
clean_country["country"] = clean_country["country"].str.replace(r'\(.*?\)', '', regex=True)
#Replace United states of mareican with USA
# clean_country["country"] = clean_country["country"].str.replace(r'(U.*S.*A)', 'USA', regex=True)


# clean_country["country"] = clean_country["country"].str.extract(r'(U.*S.*A)', 'USA')

clean_country.loc[clean_country["country"].str.contains('(U.*S)', na=False), 'country'] = "USA"

clean_country["country"] = clean_country["country"].str.replace(r'[^A-Za-z]', '', regex=True).str.strip()

# Remove any country that has more than 12 letter (i.e. longest string)
clean_country.loc[clean_country["country"].str.contains('[A-Z]{12,}', na=False), 'country'] = ""

clean_country["country"] = clean_country["country"].astype("category")
clean_country["go_out"] = clean_country["go_out"].astype("category")

C:\Users\K\AppData\Local\Temp\ipykernel_3596\2977127336.py:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  clean_country.loc[clean_country["country"].str.contains('(U.*S)', na=False), 'country'] = "USA"


In [182]:
clean_age = clean_country.copy()

clean_age.loc[clean_age["age"].str.contains('[^0-9]', na=False), 'age'] = np.nan

clean_age["age"] = clean_age["age"].astype('float')
clean_age.loc[clean_age["age"]>99, 'age'] = np.nan

99    51.0
99     NaN
99    44.0
Name: age, dtype: float64

In [ ]:
# What is the total number of candy ratings given across the three years. (Number of candy ratings, not the number of raters. Don’t count missing values)
# What was the average age of people who are going out trick or treating?
# What was the average age of people who are not going trick or treating?
# For each of joy, despair and meh, which candy bar revived the most of these ratings?
# How many people rated Starburst as despair?
# For the next three questions, count despair as -1, joy as +1, and meh as 0.

# What was the most popular candy bar by this rating system for each gender in the dataset ?
# What was the most popular candy bar in each year?
# What was the most popular candy bar by this rating for people in US, Canada, UK, and all other countries?